<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/AAL_all_devices_Prophet_models_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from fbprophet import Prophet

In [0]:
# Load the full dataset
fact_table = pd.read_csv('drive/My Drive/AAL/aal_fact_table.csv')

In [4]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,phone,tablet,wearable,others
0,2017-01-01,3558,4925,242,414
1,2017-01-02,8272,3984,544,1499
2,2017-01-03,6775,3814,501,1365
3,2017-01-04,7226,5297,460,1509
4,2017-01-05,7926,7513,468,1626


In [0]:
# Split out indivudal datasets for forecasting
phone = fact_table[['ds', 'phone']].rename(columns = {'phone': 'y'})
tablet = fact_table[['ds', 'tablet']].rename(columns = {'tablet': 'y'})
wearable = fact_table[['ds', 'wearable']].rename(columns = {'wearable': 'y'})
others = fact_table[['ds', 'others']].rename(columns = {'others': 'y'})

In [6]:
# Check the individual dataset format (testing only)
phone.head()

,ds,y
0,2017-01-01,3558
1,2017-01-02,8272
2,2017-01-03,6775
3,2017-01-04,7226
4,2017-01-05,7926


In [0]:
# Define the holidays
new_years_day = pd.DataFrame({
    'holiday': 'new_years_day',
    'ds': pd.to_datetime(['2017-01-01',
                          '2018-01-01',
                          '2019-01-01',
                          '2020-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

mlk_day = pd.DataFrame({
    'holiday': 'mlk_day',
    'ds': pd.to_datetime(['2017-01-16',
                          '2018-01-15',
                          '2019-01-21',
                          '2020-01-20']),
    'lower_window': 0,
    'upper_window': 0,
})

easter_sunday = pd.DataFrame({
    'holiday': 'easter_sunday',
    'ds': pd.to_datetime(['2017-04-16',
                          '2018-04-01',
                          '2019-04-21',
                          '2020-04-12']),
    'lower_window': 0,
    'upper_window': 0,
})

memorial_day = pd.DataFrame({
    'holiday': 'memorial_day',
    'ds': pd.to_datetime(['2017-05-29',
                          '2018-05-28',
                          '2019-05-27',
                          '2020-05-25']),
    'lower_window': 0,
    'upper_window': 0,
})

independence_day = pd.DataFrame({
    'holiday': 'independence_day',
    'ds': pd.to_datetime(['2017-07-04',
                          '2018-07-04',
                          '2019-07-04',
                          '2020-07-04']),
    'lower_window': 0,
    'upper_window': 0,
})

thanksgiving_day = pd.DataFrame({
    'holiday': 'thanksgiving_day',
    'ds': pd.to_datetime(['2017-11-23',
                          '2018-11-22',
                          '2019-11-28',
                          '2020-11-26']),
    'lower_window': -3,
    'upper_window': 0,
})

black_friday = pd.DataFrame({
    'holiday': 'black_friday',
    'ds': pd.to_datetime(['2017-11-24',
                          '2018-11-23',
                          '2019-11-29',
                          '2020-11-27']),
    'lower_window': 0,
    'upper_window': 1,
})

cyber_monday = pd.DataFrame({
    'holiday': 'cyber_monday',
    'ds': pd.to_datetime(['2017-11-27',
                          '2018-11-26',
                          '2019-12-02',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 1,
})

christmas_day = pd.DataFrame({
    'holiday': 'christmas_day',
    'ds': pd.to_datetime(['2017-12-25',
                          '2018-12-25',
                          '2019-12-25',
                          '2020-12-25']),
    'lower_window': -8,
    'upper_window': 3,
})

apple_phone_launch = pd.DataFrame({
    'holiday': 'apple_phone_launch',
    'ds': pd.to_datetime(['2017-09-22',
                          '2017-11-03',
                          '2018-09-21',
                          '2018-10-26',
                          '2019-09-20']),
    'lower_window': 0,
    'upper_window': 0,
})

samsung_phone_launch = pd.DataFrame({
    'holiday': 'samsung_phone_launch',
    'ds': pd.to_datetime(['2017-04-21',
                          '2018-03-16',
                          '2019-03-08',
                          '2020-03-06']),
    'lower_window': 0,
    'upper_window': 0,
})

samsung_note_launch = pd.DataFrame({
    'holiday': 'samsung_note_launch',
    'ds': pd.to_datetime(['2017-09-15',
                          '2018-08-24',
                          '2019-08-23']),
    'lower_window': 0,
    'upper_window': 0,
})

apple_tablet_launch = pd.DataFrame({
    'holiday': 'apple_tablet_launch',
    'ds': pd.to_datetime(['2017-03-24',
                          '2017-06-05',
                          '2018-03-27',
                          '2018-11-07',
                          '2019-03-18']),
    'lower_window': 0,
    'upper_window': 0,
})

samsung_tablet_launch = pd.DataFrame({
    'holiday': 'samsung_tablet_launch',
    'ds': pd.to_datetime(['2017-03-24',
                          '2018-08-10',
                          '2019-04-26',
                          '2019-08-30']),
    'lower_window': 0,
    'upper_window': 0,
})

apple_watch_launch = pd.DataFrame({
    'holiday': 'apple_watch_launch',
    'ds': pd.to_datetime(['2017-09-20',
                          '2018-09-19',
                          '2019-09-20']),
    'lower_window': 0,
    'upper_window': 0,
})

In [0]:
holidays_phone = pd.concat((new_years_day,
                            mlk_day,
                            easter_sunday,
                            memorial_day,
                            independence_day,
                            thanksgiving_day,
                            black_friday,
                            cyber_monday,
                            christmas_day,
                            apple_phone_launch,
                            samsung_phone_launch,
                            samsung_note_launch))

In [0]:
holidays_tablet = pd.concat((new_years_day,
                             mlk_day,
                             easter_sunday,
                             memorial_day,
                             independence_day,
                             thanksgiving_day,
                             black_friday,
                             cyber_monday,
                             christmas_day,
                             apple_tablet_launch,
                             samsung_tablet_launch))

In [0]:
holidays_wearable = pd.concat((new_years_day,
                               mlk_day,
                               easter_sunday,
                               memorial_day,
                               independence_day,
                               thanksgiving_day,
                               black_friday,
                               cyber_monday,
                               christmas_day,
                               apple_watch_launch))

In [0]:
holidays_others = pd.concat((new_years_day,
                             mlk_day,
                             easter_sunday,
                             memorial_day,
                             independence_day,
                             thanksgiving_day,
                             black_friday,
                             cyber_monday,
                             christmas_day))

In [12]:
# Define and fit the models
phone_model = Prophet(holidays = holidays_phone).fit(phone)
tablet_model = Prophet(holidays = holidays_tablet).fit(tablet)
wearable_model = Prophet(holidays = holidays_wearable).fit(wearable)
others_model = Prophet(holidays = holidays_others).fit(others)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [0]:
# NUmber of periods into the future to forecast
n = 31

In [0]:
# Create dataframes to hold predictions
phone_future = phone_model.make_future_dataframe(periods = n)
tablet_future = tablet_model.make_future_dataframe(periods = n)
wearable_future = wearable_model.make_future_dataframe(periods = n)
others_future = others_model.make_future_dataframe(periods = n)

In [0]:
# Make predictions
phone_forecast = phone_model.predict(phone_future)
tablet_forecast = tablet_model.predict(tablet_future)
wearable_forecast = wearable_model.predict(wearable_future)
others_forecast = others_model.predict(others_future)

In [0]:
# Get the datestamp and prediction columns
phone_forecast_subset = phone_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Phone'})
tablet_forecast_subset = tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Tablet'})
wearable_forecast_subset = wearable_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Wearable'})
others_forecast_subset = others_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Others'})

In [17]:
# Check the format of the subsetted forecasts (testing only)
phone_forecast_subset.tail()

,ds,Phone
1181,2020-03-27,10513.865984
1182,2020-03-28,10928.915559
1183,2020-03-29,7227.633361
1184,2020-03-30,8578.528520
1185,2020-03-31,8970.843933


In [0]:
# Combine the forecasts into one table
aal_pred_table = phone_forecast_subset
aal_pred_table.insert(2, 'Tablet', tablet_forecast_subset['Tablet'])
aal_pred_table.insert(3, 'Wearable', wearable_forecast_subset['Wearable'])
aal_pred_table.insert(4, 'Others', others_forecast_subset['Others'])

In [19]:
# Check the format of the final predictions table (testing only)
aal_pred_table.tail()

,ds,Phone,Tablet,Wearable,Others
1181,2020-03-27,10513.865984,3625.212870,4098.663076,1782.044738
1182,2020-03-28,10928.915559,3287.402790,3497.305074,1508.270443
1183,2020-03-29,7227.633361,1162.191694,2073.430103,604.545937
1184,2020-03-30,8578.528520,2334.171630,2837.091827,1319.743169
1185,2020-03-31,8970.843933,2589.658804,3259.705599,1388.965967


In [0]:
# Save the predictions table to a csv file
aal_pred_table.to_csv('drive/My Drive/AAL/aal_pred_table_v2.csv')